In [1]:
!pip install accelerate

In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [54]:
from collections import defaultdict
import random
import re

import pandas as pd

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, GenerationConfig

from tqdm.notebook import tqdm

In [18]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
random.seed(42)

# Interpreted Targeted Sentiment Analysis

**Задача:** проверить LLM (~7 млрд.) на задаче интерпретируемого таргетированного анализа тональности с различными типами промптов.

### 1. Loading Data

Загрузим имеющиеся данные.

**validation** - данные-примеры для составления few-shot промптов.

In [6]:
val_df = pd.read_csv('/kaggle/input/12121212/val.csv', index_col=0)
val_df.head(5)

,entity,pos,opinion_words,label
sentence,,,,
"Однако в четверг представитель неправительственной организации ""Сеть поддержки выборов в Зимбабве"" (Zimbabwe Election Support Network, ZESN) назвал результаты мониторинга процесса выборов ""катастрофой"" для Цвангираи.",Цвангираи,"[206, 215]",['катастрофой'],-1
"Речь идет о скандальном деле Mediaset, в рамках которого Берлускони обвинялся в масштабных финансовых махинациях.",Mediaset,"[29, 37]",['скандальном'],-1
А вот у Bruins неожиданно блещут Давид Крейчи (десять голов) и Натан Хортон (восемь шайб).,Давид Крейчи,"[33, 45]",['блещут'],1
Президенту Бразилии Дилме Руссефф грозит импичмент,Дилме Руссефф,"[20, 33]",['импичмент'],-1
"ODS в лице своего нового председателя Мартина Куба заявила, что не поддержит предлагаемого президентом технического правительства.",технического правительства,"[103, 129]",['не поддержит'],-1


In [7]:
val_df['label'].value_counts()

label
-1    28
 1    22
Name: count, dtype: int64

**test** - данные, для которых требуется узнать тональности и связанные с ней слова.

In [8]:
test_df = pd.read_csv('/kaggle/input/12121212/test.csv', index_col=0)
test_df.head(5)

,entity,pos
sentence,,
Иран поможет Афганистану в трудоустройстве населения и борьбе с наркотрафиком.,Афганистану,"[13, 24]"
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",Лимоновым,"[328, 337]"
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",Людмилой Алексеевой,"[174, 193]"
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",властями,"[367, 375]"
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",правозащитникам,"[102, 117]"


### 2. Loading Model

В качестве модели будем использовать последнюю версию OpenChat (7b).

В чистом виде, даже 2xT4 не хватает для полного инференса. Применим 8-битную квантизацию - упадет качество, но скорость и затраты по памяти улучшатся.

Загрузим LLM:

In [9]:
checkpoint = 'openchat/openchat-3.5-0106'

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
base_model_config = AutoConfig.from_pretrained(checkpoint, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=base_model_config.torch_dtype,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()

Настроим конфиг генерации:

In [10]:
generation_config = GenerationConfig.from_pretrained(checkpoint)

generation_config.do_sample = True
generation_config.max_new_tokens = 128
generation_config.temperature = 0.2
generation_config.top_k = 30
generation_config.top_p = 0.9
generation_config.repetition_penalty = 1.05

generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 32000,
  "max_length": 8192,
  "max_new_tokens": 128,
  "pad_token_id": 0,
  "repetition_penalty": 1.05,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}

In [11]:
def generate(prompt, generation_config):
    data = tokenizer(
        prompt,
        return_tensors="pt",
    )
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )

    outputs = []
    for sample_output_ids, sample_input_ids in zip(output_ids, data["input_ids"]):
        sample_output_ids = sample_output_ids[len(sample_input_ids):]
        sample_output = tokenizer.decode(sample_output_ids, skip_special_tokens=True)
        sample_output = sample_output.replace("</s>", "").strip()
        outputs.append(sample_output)

    return outputs[0]

### 3. Choosing Best Prompting Strategy

Выберем наилучшую (с точки зрения оценки "глазами") стратегию создания промпта:
1. **zero-shot**;
2. **few-shot**;
3. **few-shot + CoT**.

Напишем вспомогательные функции для извлечения нужных данных из датасетов:

In [12]:
def parse_test_sample(sample):
    """Parses test sample and extracts necessary prompt data."""
    text = sample[0]
    entity = sample[1]['entity']

    return text, entity

In [13]:
def parse_val_sample(sample):
    """Parses val sample and extracts necessary prompt data."""
    text = sample[0]
    entity = sample[1]['entity']
    opinions = eval(sample[1]['opinion_words'])
    label = sample[1]['label']

    return text, entity, opinions, label

In [14]:
def get_examples(val_df):
    """Extracts all examples from pandas DF to list."""
    examples = []

    for row in val_df.iterrows():
        examples.append(parse_val_sample(row))

    return examples

In [15]:
examples = get_examples(val_df)
len(examples)

50

Шаблон промпта возьмем из корпуса инструкций
OpenOrca_FLAN, на котором дообучался OpenChat:

In [16]:
def create_prompt(sample, examples, example_count=3, is_zero_shot=True, is_chain_of_thought=False):
    """Creates prompt for OpenChat model."""
    text, entity = parse_test_sample(sample)

    instruction = """Multi-choice question: What is the sentiment associated with the target entity in the following texts?
Choose your answer from: + negative; + positive;
Also, write one or 2 keyphrases from the text that helped you the most to determine the sentiment associated with the target entity.
{cot_directive}

Desired format:
Test keyphrases: [<comma_separated_list_of_keyphrases>]
Test label: positive/negative

{example_texts}{test_text}"""

    test_text = f"""
Test text: {text}
Test target entity: {entity}"""

    # zero-shot prompt
    if is_zero_shot:
        example_texts = ''
    # random few-shot prompt
    else:
        example_strings = []
        few_shots = random.sample(examples, example_count)

        for i, example in enumerate(few_shots):
            label = 'positive' if example[3] == 1 else 'negative'

            example_strings.append(f"""
Text {i + 1}: {example[0]}
Target entity {i + 1}: {example[1]}
Keyphrases {i + 1}: {example[2]}
Label {i + 1}: {label}
"""
            )

        example_texts = '##'.join(example_strings) + '##'

    # chain of thought prompt
    cot_directive = ''
    if is_chain_of_thought:
        cot_directive = 'While performing the task think step-by-step and justify your steps.'

    prompt = instruction.format(
        example_texts=example_texts,
        test_text=test_text,
        cot_directive=cot_directive,
    )

    return prompt

#### 3.1 Zero-shot

Попробуем не подавать никаких примеров в OpenChat:

In [22]:
random.seed(42)

count = 5
for item in test_df.iterrows():
    question = create_prompt(item, examples)
    print(question)

    messages = [{'role': 'user', 'content': question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    res = generate(prompt, generation_config)
    print(res)
    print('=' * 120)
    print()

    count -= 1
    if count == 0:
        break

Multi-choice question: What is the sentiment associated with the target entity in the following texts?
Choose your answer from: + negative; + positive;
Also, write one or 2 keyphrases from the text that helped you the most to determine the sentiment associated with the target entity.


Desired format:
Test keyphrases: [<comma_separated_list_of_keyphrases>]
Test label: positive/negative


Test text: Иран поможет Афганистану в трудоустройстве населения и борьбе с наркотрафиком.
Test target entity: Афганистану
Test keyphrases: поможет, трудоустройство населения, борьба с наркотрафиком
Test label: positive

Multi-choice question: What is the sentiment associated with the target entity in the following texts?
Choose your answer from: + negative; + positive;
Also, write one or 2 keyphrases from the text that helped you the most to determine the sentiment associated with the target entity.


Desired format:
Test keyphrases: [<comma_separated_list_of_keyphrases>]
Test label: positive/negative


#### 3.2 Few-shot

Теперь же попробуем подавать по 3 примера из validation-датасета, чтобы OpenChat опирался на них при генерации ответа:

In [23]:
random.seed(42)

count = 5
for item in test_df.iterrows():
    question = create_prompt(item, examples, is_zero_shot=False)
    print(question)

    messages = [{'role': 'user', 'content': question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    res = generate(prompt, generation_config)
    print(res)
    print('=' * 120)
    print()

    count -= 1
    if count == 0:
        break

Multi-choice question: What is the sentiment associated with the target entity in the following texts?
Choose your answer from: + negative; + positive;
Also, write one or 2 keyphrases from the text that helped you the most to determine the sentiment associated with the target entity.


Desired format:
Test keyphrases: [<comma_separated_list_of_keyphrases>]
Test label: positive/negative


Text 1: В своём фейсбук-аккаунте Ивоби назвал главу «Лиги» Маттео Сальвини «великим лидером», который привёл партию к победе.
Target entity 1: Маттео Сальвини
Keyphrases 1: ['великим лидером']
Label 1: positive
##
Text 2: Некоторые коллеги зовут канадскую писательницу «нашим Чеховым», сравнивая её с великим русским писателем, известного как автора интересных коротких рассказов.
Target entity 2: писательницу
Keyphrases 2: ['интересных коротких рассказов']
Label 2: positive
##
Text 3: Речь идет о скандальном деле Mediaset, в рамках которого Берлускони обвинялся в масштабных финансовых махинациях.
Target 

#### 3.3 Few-shot + CoT

Последний штрих: попросим OpenChat рассуждать шаг-за-шагом при генерации ответа (но рассуждения свои не писать).

In [24]:
random.seed(42)

count = 5
for item in test_df.iterrows():
    question = create_prompt(item, examples, is_zero_shot=False, is_chain_of_thought=True)
    print(question)

    messages = [{'role': 'user', 'content': question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    res = generate(prompt, generation_config)
    print(res)
    print('=' * 120)
    print()

    count -= 1
    if count == 0:
        break

Multi-choice question: What is the sentiment associated with the target entity in the following texts?
Choose your answer from: + negative; + positive;
Also, write one or 2 keyphrases from the text that helped you the most to determine the sentiment associated with the target entity.
While performing the task think step-by-step and justify your steps.

Desired format:
Test keyphrases: [<comma_separated_list_of_keyphrases>]
Test label: positive/negative


Text 1: В своём фейсбук-аккаунте Ивоби назвал главу «Лиги» Маттео Сальвини «великим лидером», который привёл партию к победе.
Target entity 1: Маттео Сальвини
Keyphrases 1: ['великим лидером']
Label 1: positive
##
Text 2: Некоторые коллеги зовут канадскую писательницу «нашим Чеховым», сравнивая её с великим русским писателем, известного как автора интересных коротких рассказов.
Target entity 2: писательницу
Keyphrases 2: ['интересных коротких рассказов']
Label 2: positive
##
Text 3: Речь идет о скандальном деле Mediaset, в рамках котор

#### 3.4 Results

В few-shot режиме формат ответа соблюдается намного чаще, чем в zero-shot (легче парсить ответ). 

Наилучшее качество - few-shot + CoT (не сильно больше просто few-shot). 

Для кодирования test_df будем использовать именно этот подход.

### 4. Encoding Test Data

Прогоним все примеры из test-файла через LLM и сохраним в него же ее ответы:

In [74]:
random.seed(42)

keyphrase_pattern = re.compile(r'.eyphrases.?.?: (.+)\n.')
label_pattern = re.compile(r'.abel.?.?: (.+)')

label_to_int = {
    'positive': 1,
    'negative': -1,
}
label_to_int = defaultdict(lambda: 1, label_to_int) # all other labels are treated as positive

keyphrases_list = []
labels = []
for item in tqdm(test_df.iterrows(), desc='Inferencing on test data'):
    question = create_prompt(item, examples, is_zero_shot=False, is_chain_of_thought=True)

    messages = [{'role': 'user', 'content': question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    res = generate(prompt, generation_config)
    
    keyphrases_list.append(keyphrase_pattern.search(res).group(1))
    label = label_pattern.search(res).group(1)
    labels.append(label_to_int[label])

Inferencing on test data: 0it [00:00, ?it/s]

In [81]:
new_test_df = test_df.copy()
new_test_df = new_test_df.assign(opinion_words=keyphrases_list, label=labels)
new_test_df

,entity,pos,opinion_words,label
sentence,,,,
Иран поможет Афганистану в трудоустройстве населения и борьбе с наркотрафиком.,Афганистану,"[13, 24]","['поможет', 'трудоустройство населения', 'борь...",1
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",Лимоновым,"[328, 337]","['отказали', 'радикально настроенной оппозиции']",-1
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",Людмилой Алексеевой,"[174, 193]","['правозащитникам', 'Триумфальной площади', 'т...",1
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",властями,"[367, 375]","['разрешили', 'отказали']",1
"Две параллельные акции протеста на площади стали возможны из-за того, что московские власти разрешили правозащитникам во главе с председателем Московской Хельсинкской группы Людмилой Алексеевой провести митинг с участием тысячи человек на Триумфальной площади 31 декабря, но отказали радикально настроенной оппозиции во главе с Лимоновым, который отверг предложенные властями варианты проведения митинга.",правозащитникам,"[102, 117]","['правозащитникам', 'разрешили']",1
...,...,...,...,...
"Летом клуб с ""Сантьяго Бернабеу"" пригласил звезду сборной Германии Месута Озила, который пришелся весьма ко двору, и который действует на схожей с бразильцем позиции.",Месута Озила,"[67, 79]","['звезда сборной Германии', 'пригласил', 'дейс...",1
Манчестер Сити продолжит борьбу за переход Кака.,Кака,"[43, 47]","['переход', 'борьба']",1
"По мнению исполнительного директора ""Сити"" Гари Кука, тогда Кака посчитал, что ""горожане"" будут для него шагом назад по сравнению с ""Миланом"".",горожане,"[80, 88]",['шагом назад'],-1


In [82]:
new_test_df.to_csv('new_test.csv')

### 5. Conclusion

Итого, OpenChat в целом неплохо справляется с задачей интерпретируемого таргетированного анализа:
- метки классифицируются почти всегда верно, однако даже явное указание в промпте, что метка может иметь только 1 из 2 зафиксированных значений, не мешает модели присуждать свои метки (mixed, neutral, и т.д.);
- словосочетания, содержащие тональность, +- похожи на истину, но иногда чересчур длинны (достаточно было 1-2 слов);
- few-shot помогает лучше придерживаться указанного в промпте формата вывода.

Помимо OpenChat, пробовал microsoft/phi-2 (2.7b) (потому что тогда еще не вспомнил про квантизацию), но она заметно хуже на этой конкретно задаче. В итоге просто взял более свежую версию OpenChat.

Инференс очень долгий, т.к. T4, к сожалению, НЕ поддерживает flash-attention-2, а возможности использовать платный тариф нет.

Как я понял, метрики реализовывать на своей стороне не нужно (т.к. не на чем модель проверять, в test-выборке нет ground-truth меток).

Потенциальные улучшения:
1. подольше поиграться с конфигом генерации;
2. попробовать knn-few-shot вместо random-few-shot.